In [9]:
import boto3 
import botocore
from creds import access_key_id,secret_access_key, bucket_name, database
import pandas as pd
from sqlalchemy import create_engine

# download file from aws s3 to local
def get_txt(s3,bucket_name, filename):
    try:
        s3.Bucket(bucket_name).download_file(filename, filename)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == '404':
            print('object does not exist')
        else:
            raise
            
# MAIN: download data files from s3, transform into one global table and write to postgres DB           
def main():
    # aws s3 authentication
    s3 = boto3.resource(
        's3',
        aws_access_key_id=access_key_id,
        aws_secret_access_key=secret_access_key,
    )

    #3. Use given credentials to download the following four CSV files from the ‘mindex-data-analytics-code-challenge’ S3 bucket.
    get_txt(s3, bucket_name,'bengals.csv')
    get_txt(s3, bucket_name,'boyd_receiving.csv')
    get_txt(s3, bucket_name,'chase_receiving.csv')
    get_txt(s3, bucket_name,'higgins_receiving.csv')

    # 4. Use the pandas library to load each CSV into its own dataframe.
    bengals = pd.read_csv('bengals.csv')
    boyd = pd.read_csv('boyd_receiving.csv')
    chase = pd.read_csv('chase_receiving.csv')
    higgins = pd.read_csv('higgins_receiving.csv')

    # 5. Join/Merge all of the dataframes together to display one global table
    # add player column to dataframes
    boyd['player'] = 'boyd'
    chase['player'] = 'chase'
    higgins['player'] = 'higgins'
    # append all player data into one dataframe
    players = pd.concat([boyd,chase,higgins]) 
    # make global table by outer join on week 
    global_table = players.merge(bengals, how='outer', on='Week')

    # 6. Replace the ‘1.0’ or ‘0.0’ values in the Result field to display ‘Win’ or ‘Loss’, respectively.
    win_loss_map = {0.0:'Loss', 1.0:'Win'}
    global_table = global_table.replace({"Result": win_loss_map})

    #7. Write the dataframe to a postgresql DB 
    engine = create_engine('postgresql://{username}:{password}@{host}:5432/{database}'.format(
                    username=database.get('username'),
                    password=database.get('password'),
                    host=database.get('host'),
                    database=database.get('database')
                    ))
    # write to postgres table
    global_table.to_sql(database.get('username'), engine,if_exists='replace')
    print('done!')

if __name__ == "__main__":
    main()


done!
